In [152]:
!pip install oracledb

In [ ]:
import oracledb
import json
import random
from datetime import datetime

In [154]:
# ------------------- Conexão com o banco -------------------

In [ ]:
def get_conn():
    with open("secret.txt", "r", encoding="utf-8") as f:
        creds = json.load(f)
    return oracledb.connect(
        user=creds["user"], 
        password=creds["password"], 
        dsn=creds["dsn"]
        )

In [156]:
# ------------------- Login e Cadastro -------------------

In [157]:
def register_user():
    conn = get_conn()
    cur = conn.cursor()

    print("\n=== CADASTRO ===")
    nome = input("Nome completo: ").strip()
    email = input("Email: ").strip()
    cpf = input("CPF: ").strip()
    senha = input("Senha: ").strip()

    cur.execute("""
        INSERT INTO users (name, email, cpf, password_hash, plan, role)
        VALUES (:1, :2, :3, :4, 'bronze', 'user')
    """, (nome, email, cpf, senha))

    conn.commit()
    print("\n✅ Usuário cadastrado com sucesso (Plano Bronze).")

    cur.close()
    conn.close()

In [158]:
def login_user():
    conn = get_conn()
    cur = conn.cursor()

    print("\n=== LOGIN ===")
    email = input("Email: ").strip()
    senha = input("Senha: ").strip()

    cur.execute("SELECT id, name, role, selected_challenge_id FROM users WHERE email=:1 AND password_hash=:2", (email, senha))
    user = cur.fetchone()
    cur.close()
    conn.close()

    if user:
        print(f"\n👋 Bem-vindo(a), {user[1]}!")
        if user[2] == "admin":
            admin_menu(user)
        else:
            user_menu(user)
    else:
        print("⚠️ Email ou senha incorretos.")

In [159]:
# MENU USUÁRIO

In [160]:
def user_menu(user):
    user_id = user[0]
    selected_challenge = user[3]

    while True:
        print("\n=== MENU USUÁRIO ===")
        print("1 - Escolher desafio")
        print("2 - Jogar desafio selecionado")
        print("3 - Ver progresso")
        print("0 - Sair")
        op = input("Escolha: ").strip()

        if op == "1":
            selected_challenge = choose_challenge(user_id)
        elif op == "2":
            if selected_challenge:
                play_selected_challenge(user_id, selected_challenge)
            else:
                print("⚠️ Nenhum desafio selecionado! Escolha um antes de jogar.")
        elif op == "3":
            view_progress(user_id)
        elif op == "0":
            print("👋 Até logo!")
            break
        else:
            print("⚠️ Opção inválida!")

In [161]:
# ESCOLHER DESAFIO

In [162]:
def choose_challenge(user_id):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT id, title FROM challenges ORDER BY id")
    desafios = cur.fetchall()

    if not desafios:
        print("⚠️ Nenhum desafio disponível.")
        return None

    print("\n=== DESAFIOS DISPONÍVEIS ===")
    for cid, title in desafios:
        print(f"[{cid}] {title}")

    escolha = input("Digite o ID do desafio que deseja escolher: ").strip()
    ids = [str(d[0]) for d in desafios]
    if escolha not in ids:
        print("⚠️ ID inválido!")
        return None

    cur.execute("UPDATE users SET selected_challenge_id = :1 WHERE id = :2", (escolha, user_id))
    conn.commit()
    cur.close()
    conn.close()

    print(f"✅ Desafio {escolha} selecionado com sucesso!")
    return int(escolha)


In [163]:
# MINI JOGOS

In [164]:
def desafio_quiz():
    perguntas = [
        ("O que é uma soft skill?", "habilidade comportamental"),
        ("O que é importante para o trabalho em equipe?", "comunicação"),
        ("O que é empatia?", "entender o outro")
    ]
    acertos = 0
    for p, resp in perguntas:
        r = input(f"{p} ").lower().strip()
        if r == resp:
            acertos += 1
    score = int((acertos / len(perguntas)) * 100)
    print(f"✅ Você acertou {acertos}/{len(perguntas)}! Pontuação: {score}%")
    return score


In [165]:
def desafio_adivinhacao():
    import random
    numero = random.randint(1, 10)
    tentativas = 0
    print("🎯 Adivinhe o número entre 1 e 10!")
    while True:
        palpite = int(input("Seu palpite: "))
        tentativas += 1
        if palpite == numero:
            print(f"✅ Acertou em {tentativas} tentativas!")
            return max(100 - tentativas * 10, 0)
        elif palpite < numero:
            print("🔼 Maior!")
        else:
            print("🔽 Menor!")

In [166]:
def desafio_digitar():
    frase = "Aprender é evoluir constantemente"
    print(f"Digite a seguinte frase:\n> {frase}")
    entrada = input("Digite: ").strip()
    if entrada == frase:
        print("✅ Perfeito!")
        return 100
    else:
        print("❌ Erro na digitação!")
        return 50

In [167]:
# JOGAR DESAFIO SELECIONADO

In [168]:
def play_selected_challenge(user_id, challenge_id):
    print(f"\n🎯 Jogando desafio {challenge_id}...\n")

    if challenge_id == 1:
        score = desafio_quiz()
    elif challenge_id == 2:
        score = desafio_adivinhacao()
    elif challenge_id == 3:
        score = desafio_digitar()
    else:
        print("⚠️ Esse desafio ainda não tem jogo implementado.")
        return

    update_progress(user_id, challenge_id, score)

In [169]:
# PROGRESSO

In [170]:
def update_progress(user_id, challenge_id, score):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("""
        INSERT INTO progress (user_id, challenge_id, score, last_update)
        VALUES (:1, :2, :3, :4)
    """, (user_id, challenge_id, score, datetime.now()))
    conn.commit()
    cur.close()
    conn.close()

In [171]:
def view_progress(user_id):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("""
        SELECT c.title, p.score, p.last_update 
        FROM progress p
        JOIN challenges c ON c.id = p.challenge_id
        WHERE p.user_id = :1
    """, (user_id,))
    rows = cur.fetchall()
    cur.close()
    conn.close()

    if not rows:
        print("⚠️ Nenhum progresso encontrado.")
    else:
        print("\n=== SEU PROGRESSO ===")
        for t, s, d in rows:
            print(f"- {t}: {s}% ({d})")


In [172]:
# MENU ADMIN

In [173]:
def admin_menu(user):
    while True:
        print("\n=== MENU ADMIN ===")
        print("1 - Criar novo desafio")
        print("2 - Ver todos os desafios")
        print("3 - Ver progresso de todos os usuários")
        print("4 - Exportar dados (JSON)")
        print("5 - Gerenciar usuários")
        print("0 - Sair")
        op = input("Escolha: ").strip()

        if op == "1":
            create_challenge()
        elif op == "2":
            list_challenges()
        elif op == "3":
            view_all_progress()
        elif op == "4":
            export_json()
        elif op == "5":
            manage_users()
        elif op == "0":
            break
        else:
            print("⚠️ Opção inválida!")

In [174]:
def create_challenge():
    conn = get_conn()
    cur = conn.cursor()
    title = input("Título do desafio: ")
    desc = input("Descrição: ")
    cur.execute("INSERT INTO challenges (title, description) VALUES (:1, :2)", (title, desc))
    conn.commit()
    cur.close()
    conn.close()
    print("✅ Desafio criado com sucesso!")

In [175]:
def list_challenges():
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT id, title FROM challenges")
    rows = cur.fetchall()
    cur.close()
    conn.close()
    print("\n=== DESAFIOS ===")
    for r in rows:
        print(f"[{r[0]}] {r[1]}")

In [176]:
def view_all_progress():
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("""
        SELECT u.name, c.title, p.score 
        FROM progress p
        JOIN users u ON u.id = p.user_id
        JOIN challenges c ON c.id = p.challenge_id
    """)
    for n, t, s in cur.fetchall():
        print(f"{n} - {t}: {s}%")
    cur.close()
    conn.close()

In [177]:
def export_json():
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT * FROM progress")
    data = [dict(zip([d[0] for d in cur.description], row)) for row in cur.fetchall()]
    with open("export_progress.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4, default=str)
    print("✅ Dados exportados para export_progress.json")
    cur.close()
    conn.close()

In [178]:
def manage_users():
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT id, name, email, role FROM users")
    users = cur.fetchall()
    for u in users:
        print(f"{u[0]} - {u[1]} ({u[2]}) [{u[3]}]")
    choice = input("Digite o ID para promover a admin ou 0 para voltar: ")
    if choice != "0":
        cur.execute("UPDATE users SET role='admin' WHERE id=:1", (choice,))
        conn.commit()
        print("✅ Usuário promovido a admin.")
    cur.close()
    conn.close()

In [179]:
# MAIN

In [180]:
def main():
    while True:
        print("\n=== SkillUp - Aprendizado Gamificado ===")
        print("1 - Registrar")
        print("2 - Login")
        print("0 - Sair")
        op = input("Escolha: ").strip()

        if op == "1":
            register_user()
        elif op == "2":
            login_user()
        elif op == "0":
            print("Até logo!")
            break
        else:
            print("⚠️ Opção inválida!")

In [181]:
if __name__ == "__main__":
    main()


=== SkillUp - Aprendizado Gamificado ===
1 - Registrar
2 - Login
0 - Sair

=== LOGIN ===

👋 Bem-vindo(a), João!

=== MENU USUÁRIO ===
1 - Escolher desafio
2 - Jogar desafio selecionado
3 - Ver progresso
0 - Sair

🎯 Jogando desafio 2...

🎯 Adivinhe o número entre 1 e 10!
🔼 Maior!
🔼 Maior!
🔼 Maior!
✅ Acertou em 4 tentativas!

=== MENU USUÁRIO ===
1 - Escolher desafio
2 - Jogar desafio selecionado
3 - Ver progresso
0 - Sair

=== SEU PROGRESSO ===
- Jogo da Adivinhação: 60% (2025-11-12 22:43:28)

=== MENU USUÁRIO ===
1 - Escolher desafio
2 - Jogar desafio selecionado
3 - Ver progresso
0 - Sair

🎯 Jogando desafio 2...

🎯 Adivinhe o número entre 1 e 10!
✅ Acertou em 1 tentativas!

=== MENU USUÁRIO ===
1 - Escolher desafio
2 - Jogar desafio selecionado
3 - Ver progresso
0 - Sair

=== SEU PROGRESSO ===
- Jogo da Adivinhação: 90% (2025-11-12 22:43:42)
- Jogo da Adivinhação: 60% (2025-11-12 22:43:28)

=== MENU USUÁRIO ===
1 - Escolher desafio
2 - Jogar desafio selecionado
3 - Ver progresso
0 - 